In [1]:
import pandas as pd
file_path = r'makemytrip_qa_dataset_mini.json'
try:
    df = pd.read_json(file_path)
    display(df.head())
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the path.")
except Exception as e:
    print(f"An error occurred: {e}")

,question,answer
0,What was the Total revenue in 2023?,"The Total revenue in 2023 was USD 593,036."
1,What was the Total revenue in 2023?,"In 2023, the company reported Total revenue of..."
2,What was the Total revenue in 2023?,"Total revenue stood at USD 593,036 in 2023."
3,What was the Total revenue in 2023?,"The company recorded USD 593,036 as Total reve..."
4,How much was the Total revenue in 2023?,"The Total revenue in 2023 was USD 593,036."


## Prepare data for fine-tuning

### Subtask:
Format the questions and answers from the JSON data into a suitable format for fine-tuning the model.

**Reasoning**:
Convert the dataframe into a list of dictionaries, where each dictionary represents a training example with the question and answer formatted as a single text string.

In [2]:
# Prepare data for fine-tuning
training_data = []

# Define a system prompt for your domain
system_prompt = "You are a helpful assistant that provides financial data from MakeMyTrip reports."

# Correctly format each training example with the chat template
for index, row in df.iterrows():
    question = row['question']
    answer = row['answer']

    # Format the data using the TinyLlama chat template
    training_data.append({
        "text": f"<|system|>\n{system_prompt}</s>\n<|user|>\n{question}</s>\n<|assistant|>\n{answer}</s>"
    })

In [3]:
# Display the first few training examples
display(training_data[:5])

[{'text': '<|system|>\nYou are a helpful assistant that provides financial data from MakeMyTrip reports.</s>\n<|user|>\nWhat was the Total revenue in 2023?</s>\n<|assistant|>\nThe Total revenue in 2023 was USD 593,036.</s>'},
 {'text': '<|system|>\nYou are a helpful assistant that provides financial data from MakeMyTrip reports.</s>\n<|user|>\nWhat was the Total revenue in 2023?</s>\n<|assistant|>\nIn 2023, the company reported Total revenue of USD 593,036.</s>'},
 {'text': '<|system|>\nYou are a helpful assistant that provides financial data from MakeMyTrip reports.</s>\n<|user|>\nWhat was the Total revenue in 2023?</s>\n<|assistant|>\nTotal revenue stood at USD 593,036 in 2023.</s>'},
 {'text': '<|system|>\nYou are a helpful assistant that provides financial data from MakeMyTrip reports.</s>\n<|user|>\nWhat was the Total revenue in 2023?</s>\n<|assistant|>\nThe company recorded USD 593,036 as Total revenue in 2023.</s>'},
 {'text': '<|system|>\nYou are a helpful assistant that provid

In [4]:
from datasets import Dataset
from transformers import AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def preprocess(example):
    # Tokenize full text
    tokens = tokenizer(example['text'], truncation=True, padding=False)
    
    # Find indices to mask
    # Everything before <|assistant|> is ignored in loss
    text = example['text']
    assistant_index = text.find("<|assistant|>")
    
    # Convert character index to token index
    prefix_ids = tokenizer(text[:assistant_index], add_special_tokens=False)['input_ids']
    prefix_len = len(prefix_ids)
    
    # Prepare labels: -100 for question/system tokens
    labels = tokens['input_ids'].copy()
    labels[:prefix_len] = [-100] * prefix_len
    
    tokens['labels'] = labels
    return tokens

dataset = Dataset.from_list(training_data)
tokenized_dataset = dataset.map(preprocess, remove_columns=["text"])

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [5]:
import os
import torch
import math
from torch import nn
from transformers import (
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

class LoraLinear(nn.Module):
    def __init__(self, in_features, out_features, r=8, lora_alpha=16, lora_dropout=0.05, bias=False):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.r = r
        self.scaling = lora_alpha / r if r > 0 else 1.0

        # Base frozen linear layer
        self.weight = nn.Parameter(torch.empty(out_features, in_features), requires_grad=False)
        self.bias = nn.Parameter(torch.zeros(out_features), requires_grad=False) if bias else None

        if r > 0:
            self.lora_A = nn.Parameter(torch.zeros((r, in_features)))
            self.lora_B = nn.Parameter(torch.zeros((out_features, r)))
            nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
            nn.init.zeros_(self.lora_B)
            self.lora_dropout = nn.Dropout(p=lora_dropout)
        else:
            self.lora_A, self.lora_B, self.lora_dropout = None, None, None

    def forward(self, x):
        # Base forward
        result = F.linear(x, self.weight, self.bias)

        # LoRA adaptation
        if self.r > 0:
            lora_out = self.lora_dropout(x) @ self.lora_A.T @ self.lora_B.T
            result = result + self.scaling * lora_out

        return result


In [8]:
# ----------------------------
# 0. Disable WandB
# ----------------------------
os.environ["WANDB_MODE"] = "disabled"

# ----------------------------
# 3. Define an EFFICIENT 4-expert MoE Layer
# ----------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F

class MoELoRALinear(nn.Module):
    def __init__(self, base_linear, r, num_experts=2, k=1, lora_alpha=16, lora_dropout=0.05):
        super().__init__()
        self.base_linear = base_linear  # <-- frozen pretrained weight
        self.num_experts = num_experts
        self.k = k

        self.experts = nn.ModuleList([
            LoraLinear(   # LoRA adapter only
                in_features=base_linear.in_features,
                out_features=base_linear.out_features,
                r=r,
                lora_alpha=lora_alpha,
                lora_dropout=lora_dropout
            )
            for _ in range(num_experts)
        ])

        self.gate = nn.Linear(base_linear.in_features, num_experts)

    def forward(self, x):
        # keep frozen pretrained path
        base_out = self.base_linear(x)

        # gating for experts
        gate_scores = torch.softmax(self.gate(x), dim=-1)

        expert_out = 0
        for i, expert in enumerate(self.experts):
            expert_out += gate_scores[..., i:i+1] * expert(x)

        return base_out + expert_out

def replace_proj_with_moe_lora(model, r=8, num_experts=2, k=1, lora_alpha=16, lora_dropout=0.05):
    """
    Replace only up_proj, down_proj, o_proj in each MLP with MoE(LoRA) versions.
    """
    for layer in model.model.layers:
        for proj_name in ["up_proj", "down_proj"]:
            old = getattr(layer.mlp, proj_name)
            moe = MoELoRALinear(
                base_linear=old,
                r=r,
                num_experts=num_experts,
                k=k,
                lora_alpha=lora_alpha,
                lora_dropout=lora_dropout,
            ).to(next(old.parameters()).device)
            setattr(layer.mlp, proj_name, moe)

    return model


# ----------------------------
# 4. Load base model with 4-bit quantization
# ----------------------------
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",
    trust_remote_code=True,
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1


# ----------------------------
# 5. Apply PEFT / LoRA adapters
# ----------------------------
# Prepare model for k-bit training
base_model = prepare_model_for_kbit_training(base_model)

model = replace_proj_with_moe_lora(
    base_model,
    r=8,
    num_experts=1,
    k=1,
    lora_alpha=16,
    lora_dropout=0.05
)

In [9]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["o_proj"],  # or o_proj too if exists
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

In [10]:
# Test the fine-tuned model
baseline_responses = []

# Define the system prompt used during fine-tuning
system_prompt = "You are a helpful assistant that provides financial data from MakeMyTrip reports."

for index, row in df.head(5).iterrows():
    question = row['question']

    # Create the message list for the chat template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    # Apply the chat template to format the input
    input_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True # This adds the <|assistant|> token at the end
    )

    # Tokenize the formatted input
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)

    # Decode the entire generated output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the generated answer part
    try:
        # The response will look like "<|system|>\n...</s>\n<|user|>\n...</s>\n<|assistant|>\n...answer...</s>"
        # We need to find the <|assistant|> token and get everything after it
        answer_start_token = '<|assistant|>'
        answer_start_index = decoded_output.rfind(answer_start_token)

        if answer_start_index != -1:
            generated_answer = decoded_output[answer_start_index + len(answer_start_token):].strip()
            # The model might generate a final </s> token, which we should remove
            if generated_answer.endswith('</s>'):
                generated_answer = generated_answer[:-len('</s>')].strip()
        else:
            generated_answer = "Could not extract answer from model output."

    except Exception as e:
        generated_answer = f"An error occurred: {e}"

    baseline_responses.append({"question": question, "generated_answer": generated_answer})

# Display the first few generated responses
display(baseline_responses[:5])

[{'question': 'What was the Total revenue in 2023?',
  'generated_answer': "I don't have access to the latest financial data of makemytrip. However, according to the company's website, the total revenue for the financial year 2023 is not mentioned. It is only mentioned that the"},
 {'question': 'What was the Total revenue in 2023?',
  'generated_answer': "I don't have access to the latest financial data of makemytrip. However, according to the company's website, the total revenue for the financial year 2023 is not mentioned. It is only mentioned that the"},
 {'question': 'What was the Total revenue in 2023?',
  'generated_answer': "I don't have access to the latest financial data of makemytrip. However, according to the company's website, the total revenue for the financial year 2023 is not mentioned. It is only mentioned that the"},
 {'question': 'What was the Total revenue in 2023?',
  'generated_answer': "I don't have access to the latest financial data of makemytrip. However, accor

In [ ]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,d} || Total params: {total:,d} || "
          f"Trainable%: {100 * trainable / total:.4f}")

# ----------------------------
# 8. Gradient checkpointing
# ----------------------------
model.config.use_cache = False
model.gradient_checkpointing_disable()

# ----------------------------
# 9. Prepare collator
# ----------------------------
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# ----------------------------
# 10. Training arguments
# ----------------------------
training_args = TrainingArguments(
    learning_rate=5e-5,
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=1, # Keep batch size small
    gradient_accumulation_steps=4, # Increased gradient accumulation steps
    logging_steps=1,
    save_steps=10,
    save_total_limit=2,
    fp16=True, # fp16 and bf16 are mutually exclusive. bf16 is recommended for Ampere+ GPUs.
    bf16=False,  # Use bf16 for better performance with 4-bit models
)

# ----------------------------
# 11. Trainer
# ----------------------------

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# ----------------------------
# 12. Train
# ----------------------------
trainer.train()

Trainable params: 3,424,256 || Total params: 1,126,710,316 || Trainable%: 0.3039


Step,Training Loss
1,2.138800
2,2.063300
3,2.098100
4,2.026800
5,1.999300
6,1.918100
7,1.929300
8,1.824500
9,1.836600
10,1.784400


In [ ]:
# Test the fine-tuned model
fine_tuned_responses = []

# Define the system prompt used during fine-tuning
system_prompt = "You are a helpful assistant that provides financial data from MakeMyTrip reports."

for index, row in df.iterrows():
    question = row['question']

    # Create the message list for the chat template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    # Apply the chat template to format the input
    input_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True # This adds the <|assistant|> token at the end
    )

    # Tokenize the formatted input
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)

    # Decode the entire generated output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the generated answer part
    try:
        # The response will look like "<|system|>\n...</s>\n<|user|>\n...</s>\n<|assistant|>\n...answer...</s>"
        # We need to find the <|assistant|> token and get everything after it
        answer_start_token = '<|assistant|>'
        answer_start_index = decoded_output.rfind(answer_start_token)

        if answer_start_index != -1:
            generated_answer = decoded_output[answer_start_index + len(answer_start_token):].strip()
            # The model might generate a final </s> token, which we should remove
            if generated_answer.endswith('</s>'):
                generated_answer = generated_answer[:-len('</s>')].strip()
        else:
            generated_answer = "Could not extract answer from model output."

    except Exception as e:
        generated_answer = f"An error occurred: {e}"

    fine_tuned_responses.append({"question": question, "generated_answer": generated_answer})

# Display the first few generated responses
display(fine_tuned_responses[:5])

In [ ]:
display(fine_tuned_responses)

In [ ]:
base_model